# Library 

In [1]:
import numpy as np
from selenium import webdriver
from time import sleep
import re
from pymongo import MongoClient
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import random
from selenium.common.exceptions import NoSuchElementException , ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.chrome.options import Options

# Class Crawl

In [2]:
class batdongsan():
    def __init__(self,link,name=[]):
        self.link = link
        self.name = name
        self.result = pd.DataFrame(columns=name)
        self.next_page=[]
        options= Options()
        options.page_load_strategy = 'eager'
        options.headless = True
        options.add_argument(r"--user-data-dir=C:\\Users\\USER\AppData\\Local\\Google\\Chrome\\User Data\Default")
        options.add_argument("--disable-javascript")
        options.add_argument("--blink-settings=imagesEnabled=false")
        self.driver = webdriver.Chrome(options=options)
    def Crawl(self):
        web_page=self.link
        while True :
            List_link , next_page = self.get_links(web_page)
            for i in range(len(List_link)-1):
                # if List_link[i] in link_have :
                #     continue
                self.getData(List_link[i])
            web_page = next_page
            if web_page == None:
                break
    def get_links(self,link):
        driver = self.driver
        driver.get(link)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
            'a.js__product-link-for-product-id'))
        )

        text = driver.find_elements(By.CSS_SELECTOR,
        'a.js__product-link-for-product-id')
        links= [i.get_attribute('href') for i in text]

        next_page = driver.find_elements(By.CSS_SELECTOR,'a.re__pagination-icon')[-1]
        next_page = next_page.get_attribute('href')
        self.next_page.append(next_page)
        return links  ,next_page
    def getData(self,link):
        driver = self.driver
        test = pd.Series(index=self.name)
        driver.get(link)


        # Các nội dung tổng quan về mẫu
        test['Link'] = link
        test['Title'] = driver.find_element(By.CSS_SELECTOR,'h1.re__pr-title').text
        test['Address'] = driver.find_element(By.CSS_SELECTOR,'.js__pr-address').text

        # Thử lấy dữ liệu xác thực và số zalo liên hệ
        try:
            test['Check'] = driver.find_element(By.CSS_SELECTOR,
            '.js__product-detail-web .re__pr-stick-listing-verified .re__text').text
            
        except NoSuchElementException :
            test['Check']='Chưa Xác Thực'
        # Thử lấy dữ liệu chi tiêt về mẫu
        name_data = driver.find_elements(By.CSS_SELECTOR,

        '.js__li-specs  .js__section-body .js__other-info .re__pr-specs-content-item .re__pr-specs-content-item-title')
        value_data =  driver.find_elements(By.CSS_SELECTOR,

        '.js__li-specs  .js__section-body .js__other-info .re__pr-specs-content-item .re__pr-specs-content-item-value')
        data = pd.Series(value_data,index=name_data)

        for i in range(len(name_data)):
            test[name_data[i].text] = value_data[i].text

        name_data_2 = driver.find_elements(By.CSS_SELECTOR,
        '.js__pr-config .js__pr-config-item .title')

        value_data_2 = driver.find_elements(By.CSS_SELECTOR,
        '.js__pr-config .js__pr-config-item .value')

        for i in range(len(name_data_2)):
            test[name_data_2[i].text] = value_data_2[i].text

        self.result = pd.concat([self.result, test.to_frame().T], ignore_index=True)


# Crawl Data

- Infomation before crawl

In [3]:
nameCol_Maximum = ['Link','Diện tích', 'Mức giá','Hướng nhà','Hướng ban công', 'Số tầng', 'Số phòng ngủ', 'Số toilet', 'Pháp lý',
 'Nội thất','Title','Address','Ngày đăng','Ngày hết hạn','Loại tin','Mã tin','Check','Đường vào','Mặt tiền']
nameCol = ['Loại tin','Mã tin','Link','Diện tích', 'Mức giá','Hướng nhà', 'Số tầng', 'Số phòng ngủ', 'Số toilet', 'Pháp lý',
 'Nội thất','Title','Address','Đường vào','Mặt tiền']
# link_web = 'https://batdongsan.com.vn/ban-nha-dat-tp-hcm'
# link_web = 'https://batdongsan.com.vn/ban-can-ho-chung-cu-tp-hcm'
# link_web = 'https://batdongsan.com.vn/ban-nha-rieng-tp-hcm'
# link_web = 'https://batdongsan.com.vn/ban-dat-tp-hcm'
link_web = 'https://batdongsan.com.vn/ban-nha-mat-pho-tp-hcm'
# link_web = 'https://batdongsan.com.vn/ban-nha-biet-thu-lien-ke-tp-hcm'


- Create Class

In [4]:
X = batdongsan(link=link_web,name=nameCol)

- Crawl Data

In [5]:
X.Crawl()

C:\Users\USER\AppData\Local\Temp\ipykernel_11436\2664304168.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://batdongsan.com.vn/ban-nha-mat-pho-duong-nguyen-ngoc-phuong_1-phuong-19/ban-tien-kinh-doanh-view-so-thu-pr41475066' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test['Link'] = link
C:\Users\USER\AppData\Local\Temp\ipykernel_11436\2664304168.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'https://batdongsan.com.vn/ban-nha-mat-pho-duong-truong-sa-phuong-13-1/ban-goc-3-tien-quan-highlands-coffee-964-13-quan-3-pr41504098' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test['Link'] = link
C:\Users\USER\AppData\Local\Temp\ipykernel_11436\2664304168.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in 

TimeoutException: Message: timeout: Timed out receiving message from renderer: 282.186
  (Session info: chrome=130.0.6723.117)
Stacktrace:
	GetHandleVerifier [0x00007FF7424F38A5+3004357]
	(No symbol) [0x00007FF742189970]
	(No symbol) [0x00007FF74203582A]
	(No symbol) [0x00007FF742023C07]
	(No symbol) [0x00007FF74202390A]
	(No symbol) [0x00007FF74202153E]
	(No symbol) [0x00007FF742021FEF]
	(No symbol) [0x00007FF7420308EE]
	(No symbol) [0x00007FF7420438AD]
	(No symbol) [0x00007FF74204922A]
	(No symbol) [0x00007FF742022754]
	(No symbol) [0x00007FF742043691]
	(No symbol) [0x00007FF7420CBD91]
	(No symbol) [0x00007FF7420AB983]
	(No symbol) [0x00007FF742077628]
	(No symbol) [0x00007FF742078791]
	GetHandleVerifier [0x00007FF74251A00D+3161901]
	GetHandleVerifier [0x00007FF74256E060+3506048]
	GetHandleVerifier [0x00007FF74256400D+3465005]
	GetHandleVerifier [0x00007FF7422E0EEB+830987]
	(No symbol) [0x00007FF74219467F]
	(No symbol) [0x00007FF7421909D4]
	(No symbol) [0x00007FF742190B6D]
	(No symbo

# Store Data

- MongoDB

In [3]:
client = MongoClient("mongodb://localhost:27017/")
db = client["batdongsan"]
collection = db["batdongsan_com_vn"]
data=df
data_dict = data.to_dict("records")
collection.insert_many(data_dict)

InsertManyResult([ObjectId('6733f91471202be5d72b1d4c'), ObjectId('6733f91471202be5d72b1d4d'), ObjectId('6733f91471202be5d72b1d4e'), ObjectId('6733f91471202be5d72b1d4f'), ObjectId('6733f91471202be5d72b1d50'), ObjectId('6733f91471202be5d72b1d51'), ObjectId('6733f91471202be5d72b1d52'), ObjectId('6733f91471202be5d72b1d53'), ObjectId('6733f91471202be5d72b1d54'), ObjectId('6733f91471202be5d72b1d55'), ObjectId('6733f91471202be5d72b1d56'), ObjectId('6733f91471202be5d72b1d57'), ObjectId('6733f91471202be5d72b1d58'), ObjectId('6733f91471202be5d72b1d59'), ObjectId('6733f91471202be5d72b1d5a'), ObjectId('6733f91471202be5d72b1d5b'), ObjectId('6733f91471202be5d72b1d5c'), ObjectId('6733f91471202be5d72b1d5d'), ObjectId('6733f91471202be5d72b1d5e'), ObjectId('6733f91471202be5d72b1d5f'), ObjectId('6733f91471202be5d72b1d60'), ObjectId('6733f91471202be5d72b1d61'), ObjectId('6733f91471202be5d72b1d62'), ObjectId('6733f91471202be5d72b1d63'), ObjectId('6733f91471202be5d72b1d64'), ObjectId('6733f91471202be5d72b1d

- PostgresSQL

In [ ]:
from sqlalchemy import create_engine
data = X.result
# Thông tin kết nối PostgreSQL
db_user = "postgres"
db_password = "ky1213141516"
db_host = "localhost"
db_port = "3000"
db_name = "postgres"

# Tạo kết nối với PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# Lưu DataFrame vào PostgreSQL (chọn tên bảng phù hợp)
data.to_sql('batdongsan', engine, if_exists='replace', index=False)

23